In [1]:
from matplotlib import font_manager, rc  # 한글사용 matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import codecs
import numpy as np
import datetime
import json
import tensorflow as tf
import jpype
from konlpy.tag import Okt
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame as df
InteractiveShell.ast_node_interactivity = 'all'
os.chdir("C:/Users/windd/pythondata")
%matplotlib inline
font_name = font_manager.FontProperties(
    fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_afsnt = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT.csv',engine='python',encoding='cp949')
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

df_afsnt.head()

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,sched_time,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,...,10:05:00,10:32:00,N,NaN,N,NaN,2017-01-01,2017-01-01 10:05:00,2017-01-01 10:32:00,27
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,...,09:30:00,09:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 09:30:00,2017-01-01 09:31:00,1
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,...,12:45:00,13:03:00,N,NaN,N,NaN,2017-01-01,2017-01-01 12:45:00,2017-01-01 13:03:00,18
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,...,13:25:00,14:09:00,Y,C02,N,NaN,2017-01-01,2017-01-01 13:25:00,2017-01-01 14:09:00,44
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,...,16:10:00,16:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 16:10:00,2017-01-01 16:31:00,21


In [3]:
df_afsnt_added = pd.read_csv('bigcontest/링크_시퀀스_혼잡도.csv', engine = 'python')

In [4]:
# 공항 테이블
df_afsnt_added['origin'] = df_afsnt_added['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
df_afsnt_added['dest'] = df_afsnt_added['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})

# 항공사 테이블 
df_afsnt_added['airline'] = df_afsnt_added['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})

In [5]:
len(df_afsnt_added)

979436

In [6]:
# 링크테이블

df_afsnt_added_d = df_afsnt_added[df_afsnt_added['is_arrive']=='D']

link_table_d = df_afsnt_added_d.pivot_table(index=['origin','dest','link'],aggfunc='count',values='is_arrive')
link_table_d = link_table_d.reset_index(['origin','dest','link'])
link_table_d.columns = ['origin','dest','link','count']

df_afsnt_added_a = df_afsnt_added[df_afsnt_added['is_arrive']=='A']

link_table_a = df_afsnt_added_a.pivot_table(index=['origin','dest','link'],aggfunc='count',values='is_arrive')
link_table_a = link_table_a.reset_index(['origin','dest','link'])
link_table_a.columns = ['origin','dest','link','count']

In [7]:
# 출발일 경우
# 고민사항
# 현재 링크데이터는 출발과 도착으로 나뉘었다. 
# 원래 동일한게 맞지만 출도착 쌍이 맞지 않는 데이터가 있기 때문에 다르며
# 중심성 분석을 위해서 출발하는 경우를 대상으로 진행했음

In [8]:
# 중심성 분석 함수
# 가중치 고려

def get_degree_centrality(df,arp) :
    N = 15
    Z = df[df['origin']==arp]['count'].sum()
    D = Z/(N-1)
    return D

In [9]:
# 가중치 고려 x

def degree_centrality(df,arp):
    N =15
    a = df[df['origin']==arp]
    D = len((a['origin']+','+a['dest']).unique())/(N-1)
    return D

In [8]:
# 거리중심성 분석
# 어떤 테이블이 있을건데 이것은 출발 - 도착 - 거리를 보여주는 데이터프레임 = df_distance 

origin_dest_unique = (df_afsnt_added['origin']+','+df_afsnt_added['dest']).unique()

def get_distance_centrality(origin):
    N = 15
    df_distance2 = df_distance[df_distance['origin']==origin]
    df_distance2.index = list(range(len(df_distance2)))
    distance_sum = 0
    for i in range(len(df_distance2)):
        if (df_distance2['origin'][i]+','+df_distance2['dest'][i]) in origin_dest_unique :
            distance_sum += df_distance2['distance']
            
    distance_sum_inverse = (df_distance_sum)**(-1)
    distance_center = (N-1)*distacne_sum_inverse
    return distance_center     

In [119]:
# 거리 중심성 분석

arp_list = list(link_table_d['origin'].unique())
center_distance = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_distance_centrality(i)
    D_list.append(D)

center_distance['center_distance'] = D_list

center_distance.sort_values('center',ascending = False)

NameError: name 'get_distance_centrality' is not defined

In [10]:
len(((df_afsnt_added['origin']+','+df_afsnt_added['dest']).unique()))

84

In [11]:
# 중심성 분석 

arp_list = list(link_table_d['origin'].unique())
center_link = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table_d, i)
    D_list.append(D)

center_link['degree_center'] = D_list

center_link.sort_values('degree_center',ascending = False)

,origin,degree_center
12,제주,13959.428571
2,김포,10998.500000
3,김해,4305.428571
13,청주,1225.357143
4,대구,1194.928571
0,광주,1175.571429
9,울산,569.285714
11,인천,477.142857
8,여수,447.714286
6,사천,169.928571


In [12]:
# 가중치 고려x

N =15
arp_list = list(link_table_d['origin'].unique())
center_link2= pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table_d, i)
    D_list.append(D)
    
center_link2['degree_center2'] = D_list
center_link2.sort_values(by='degree_center2',ascending=False)

,origin,degree_center2
2,김포,1.000000
12,제주,0.857143
3,김해,0.642857
0,광주,0.357143
4,대구,0.357143
5,무안,0.357143
13,청주,0.357143
9,울산,0.285714
11,인천,0.285714
6,사천,0.214286


In [13]:
center_link_total = pd.merge(center_link, center_link2)
center_link_total

,origin,degree_center,degree_center2
0,광주,1175.571429,0.357143
1,군산,146.642857,0.142857
2,김포,10998.500000,1.000000
3,김해,4305.428571,0.642857
4,대구,1194.928571,0.357143
5,무안,112.000000,0.357143
6,사천,169.928571,0.214286
7,양양,0.428571,0.214286
8,여수,447.714286,0.214286
9,울산,569.285714,0.285714


In [14]:
ref_dict = {}

for i in range(len(center_link_total)):
    ref_dict[center_link_total['origin'][i]] = center_link_total['degree_center'][i]
    
ref_dict2 = {}
for i in range(len(center_link_total)):
    ref_dict2[center_link_total['origin'][i]] = center_link_total['degree_center2'][i]

In [15]:
df_afsnt_added2 = df_afsnt_added

df_afsnt_added2['degree_center'] = df_afsnt_added2['origin'].map(ref_dict)
df_afsnt_added2['degree_center2'] = df_afsnt_added2['origin'].map(ref_dict2)

In [16]:
df_afsnt_added2[['date','origin','dest','tailnum','sched_datetime','degree_center','degree_center2']]

,date,origin,dest,tailnum,sched_datetime,degree_center,degree_center2
0,2017-01-01 00:00:00,김포,제주,SEw3Nzc2,2017-01-01 06:10:00,10998.500000,1.000000
1,2017-01-01 00:00:00,김포,제주,SEw4MjM2,2017-01-01 06:15:00,10998.500000,1.000000
2,2017-01-01 00:00:00,김포,제주,SEw4MjM3,2017-01-01 06:20:00,10998.500000,1.000000
3,2017-01-01 00:00:00,김포,제주,SEw4MjA3,2017-01-01 06:25:00,10998.500000,1.000000
4,2017-01-01 00:00:00,김포,제주,SEw4MDMx,2017-01-01 06:30:00,10998.500000,1.000000
5,2017-01-01 00:00:00,김포,제주,SEw3NTYz,2017-01-01 06:30:00,10998.500000,1.000000
6,2017-01-01 00:00:00,김포,제주,SEw4MDE1,2017-01-01 06:40:00,10998.500000,1.000000
7,2017-01-01 00:00:00,김포,제주,SEw3NTk0,2017-01-01 06:40:00,10998.500000,1.000000
8,2017-01-01 00:00:00,김포,제주,SEw3Nzcz,2017-01-01 06:45:00,10998.500000,1.000000
9,2017-01-01 00:00:00,김포,제주,SEw3Nzg1,2017-01-01 06:45:00,10998.500000,1.000000


In [69]:
distance = pd.read_csv('bigcontest/airport_distance.csv', encoding = 'cp949')

In [70]:
distance

,origin,dest,distance_mile,distance_km
0,인천,김해,200.56,322.77
1,인천,대구,152.93,246.11
2,인천,무안,171.14,275.43
3,인천,울산,195.40,314.46
4,인천,제주,272.97,439.31
5,인천,청주,67.31,108.33
6,광주,김포,168.61,271.36
7,광주,제주,112.93,181.74
8,광주,인천,161.15,259.34
9,광주,무안,25.62,41.23


In [71]:
distance['or_dest'] = distance['origin']+','+distance['dest']
df_afsnt_added2['origin_dest'] = df_afsnt_added2['origin']+','+df_afsnt_added2['dest']

In [72]:
ref_dict = {}

for i in range(len(distance)):
    ref_dict[distance['or_dest'][i]] = distance['distance_mile'][i]
    
ref_dict2 = {}

for i in range(len(distance)):
    ref_dict2[distance['or_dest'][i]] = distance['distance_km'][i]

df_afsnt_added2['distance_mile'] = df_afsnt_added2['origin_dest'].map(ref_dict)
df_afsnt_added2['distance_km'] = df_afsnt_added2['origin_dest'].map(ref_dict2)

In [76]:
len(distance[distance['origin']=='김포']['dest'])

14